![MLU Logo](../../data/MLU_Logo.png)

## Amazon Access Samples Data Set
 
 Let's apply our boosting algorithm to a real dataset! We are going to use the __Amazon Access Samples dataset__. 
 
 We download this dataset from UCI ML repository from this [link](https://archive.ics.uci.edu/ml/datasets/Amazon+Access+Samples). Dua, D. and Graff, C. (2019). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.

 
__Dataset description:__

Employees need to request certain resources to fulfill their daily duties. This data consists of anonymized historical data of employee IT access requests. Data fields look like this:
 #### Column Descriptions

* __ACTION__: 1 if the resource was approved, 0 if not.
* __RESOURCE__: An ID for each resource
* __PERSON_MGR_ID__: ID of the user's manager
* __PERSON_ROLLUP_1__: User grouping ID
* __PERSON_ROLLUP_2__: User grouping ID
* __PERSON_BUSINESS_TITLE__: Title ID 
* __PERSON_JOB_FAMILY__: Job family ID 
* __PERSON_JOB_CODE__: Job code ID 

Our task is to build a machine learning model that can automatically provision an employee's access to company resources given employee profile information and the resource requested.

### 1. Download and process the dataset

In this section, we will download our dataset and process it. It consists of two files, we will run the following code cells to get our dataset as a single file at the end. One of the files is large (4.8GB), so make sure you have enough storage.

In [2]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00216/amzn-anon-access-samples.tgz

zsh:1: command not found: wget


In [ ]:
! tar -zxvf amzn-anon-access-samples.tgz

We have the following files:
* __amzn-anon-access-samples-2.0.csv__: Employee profile data.
* __amzn-anon-access-samples-history-2.0.csv__: Resource provision history

Below, we first read the amzn-anon-access-samples-2.0.csv file (it is a large file) and use some employee fields.

In [3]:
import pandas as pd
import random 

person_fields = ["PERSON_ID", "PERSON_MGR_ID",
                 "PERSON_ROLLUP_1", "PERSON_ROLLUP_2",
                 "PERSON_DEPTNAME", "PERSON_BUSINESS_TITLE",
                 "PERSON_JOB_FAMILY", "PERSON_JOB_CODE"]

people = {}
for chunk in pd.read_csv('amzn-anon-access-samples-2.0.csv', usecols = person_fields, chunksize=5000): 
    for index, row in chunk.iterrows():
        people[row["PERSON_ID"]] = [row["PERSON_MGR_ID"], row["PERSON_ROLLUP_1"],
                                    row["PERSON_ROLLUP_2"], row["PERSON_DEPTNAME"],
                                    row["PERSON_BUSINESS_TITLE"], row["PERSON_JOB_FAMILY"],
                                    row["PERSON_JOB_CODE"]]

FileNotFoundError: [Errno 2] No such file or directory: 'amzn-anon-access-samples-2.0.csv'

Now, let's read the resource provision history file. Here, we will create our dataset. We will read the add access and remove access actions and save them.

In [4]:
add_access_data = []
remove_access_data = []

df = pd.read_csv('amzn-anon-access-samples-history-2.0.csv')

# Loop through unique logins (employee ids)
for login in df["LOGIN"].unique():
    login_df = df[df["LOGIN"]==login].copy()
    # Save actions
    for target in login_df["TARGET_NAME"].unique():
        login_target_df = login_df[login_df["TARGET_NAME"]==target]
        unique_actions = login_target_df["ACTION"].unique()
        if((len(unique_actions)==1) and (unique_actions[0]=="remove_access")):
            remove_access_data.append([0, target] + people[login])
        elif((len(unique_actions)==1) and (unique_actions[0]=="add_access")):
            add_access_data.append([1, target] + people[login])

# Create random seed
random.seed(30)

# We will use only 8000 random add_access data
add_access_data = random.sample(add_access_data, 8000)

# Add them together
data = add_access_data + remove_access_data

# Let's shuffle it
random.shuffle(data)

Let's save this data so that we can use it later

In [5]:
df = pd.DataFrame(data, columns=["ACTION", "RESOURCE",
                                 "MGR_ID", "ROLLUP_1",
                                 "ROLLUP_2", "DEPTNAME",
                                 "BUSINESS_TITLE", "JOB_FAMILY",
                                 "JOB_CODE"])

df.to_csv("data.csv", index=False)

Here is how our data look like:

In [6]:
df.head()

,ACTION,RESOURCE,MGR_ID,ROLLUP_1,ROLLUP_2,DEPTNAME,BUSINESS_TITLE,JOB_FAMILY,JOB_CODE
0,1,9802,43122,2,3,33467,45383,11,33326
1,1,10617,36504,33416,33689,36505,41299,33430,33326
2,1,9446,35624,33316,34256,35625,41014,33461,33326
3,1,11065,34326,33299,34397,38458,38459,33678,33289
4,1,11149,40640,33283,40641,40642,40643,33291,33431


In [7]:
# Delete the downloaded files
! rm amzn-anon-access-samples-2.0.csv amzn-anon-access-samples-history-2.0.csv amzn-anon-access-samples.tgz

### 2. LightGBM

Let's use LightGBM on this dataset. 

In [5]:
! pip install -q lightgbm

scikit-learn 0.20.4 has requirement scipy>=0.13.3, but you'll have scipy 0.13.0b1 which is incompatible.
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/Library/Python/2.7/site-packages/scikit_learn-0.20.4.dist-info'
Consider using the `--user` option or check the permissions.



Let's read the dataset

In [9]:
import pandas as pd
import numpy as np

data = pd.read_csv("data.csv")

In [10]:
data.head()

,ACTION,RESOURCE,MGR_ID,ROLLUP_1,ROLLUP_2,DEPTNAME,BUSINESS_TITLE,JOB_FAMILY,JOB_CODE
0,1,9802,43122,2,3,33467,45383,11,33326
1,1,10617,36504,33416,33689,36505,41299,33430,33326
2,1,9446,35624,33316,34256,35625,41014,33461,33326
3,1,11065,34326,33299,34397,38458,38459,33678,33289
4,1,11149,40640,33283,40641,40642,40643,33291,33431


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8152 entries, 0 to 8151
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   ACTION          8152 non-null   int64
 1   RESOURCE        8152 non-null   int64
 2   MGR_ID          8152 non-null   int64
 3   ROLLUP_1        8152 non-null   int64
 4   ROLLUP_2        8152 non-null   int64
 5   DEPTNAME        8152 non-null   int64
 6   BUSINESS_TITLE  8152 non-null   int64
 7   JOB_FAMILY      8152 non-null   int64
 8   JOB_CODE        8152 non-null   int64
dtypes: int64(9)
memory usage: 573.3 KB


In [12]:
data["ACTION"].value_counts()

1    8000
0     152
Name: ACTION, dtype: int64

We will fix the column types below to make sure they are handled as categorical variables.

In [13]:
from sklearn.model_selection import train_test_split

y = data["ACTION"].values
X = data.drop(columns='ACTION')

for c in X.columns:
    X[c] = X[c].astype('category')
    
X_train, X_valid, y_train, y_valid = train_test_split(X,
                                                      y,
                                                      test_size=0.15,
                                                      random_state=136,
                                                      stratify=y
                                                     )

Let's fit the lightGBM model below.

In [14]:
import lightgbm as lgb

# Create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

# Let's see our parameters

# boosting_type (string, optional (default='gbdt'))
# ‘gbdt’, traditional Gradient Boosting Decision Tree.
# ‘dart’, Dropouts meet Multiple Additive Regression Trees.
# ‘goss’, Gradient-based One-Side Sampling.
# ‘rf’, Random Forest.

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary', # ‘regression’ for LGBMRegressor, ‘binary’ or ‘multiclass’ for LGBMClassifier
    'metric': ['auc'],
    'n_estimators': 50, # We can change it, by default 100
    'learning_rate': 0.1, # Default 0.1
    'num_iterations': 1000, # Default 100
    'is_unbalance': True, # Used to fix the class imbalance in the dataset
    'verbose': 1
}

#Train
gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                early_stopping_rounds=20
               )

[LightGBM] [Info] Number of positive: 6800, number of negative: 129
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000374 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2538
[LightGBM] [Info] Number of data points in the train set: 6929, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.981383 -> initscore=3.964865
[LightGBM] [Info] Start training from score 3.964865
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 0.766304
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 0.850851
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's auc: 0.849565
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's auc: 0.856069
[LightGBM] [Warning] No further split

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Let's see the overall performance on validation set.

In [15]:
from sklearn.metrics import classification_report

y_pred = gbm.predict(X_valid, num_iteration=gbm.best_iteration)

print(classification_report(y_valid, np.round(y_pred)))

              precision    recall  f1-score   support

           0       0.26      0.61      0.37        23
           1       0.99      0.97      0.98      1200

    accuracy                           0.96      1223
   macro avg       0.63      0.79      0.67      1223
weighted avg       0.98      0.96      0.97      1223

